In [1]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from tqdm import tqdm  # For showing the progress bar

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import torch
print(torch.cuda.is_available())


False


In [5]:
import torch
print(torch.__version__)  # Should print the installed PyTorch version
print(torch.cuda.is_available())  # Should return True if GPU is available, False otherwise


2.6.0+cpu
False


In [7]:
!pip install tqdm


In [6]:
import tqdm
print("tqdm is installed successfully!")


tqdm is installed successfully!


In [7]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [13]:
# Load full dataset and split into train and validation sets
dataset = datasets.ImageFolder(root='dataset/preprocessedDataset', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [22]:
# Load data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
# Define the CNN model
class PestDetectionCNN(nn.Module):
    def __init__(self, num_classes):
        super(PestDetectionCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize model
num_classes = len(dataset.classes)
model = PestDetectionCNN(num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy
def calculate_accuracy(y_true, y_pred):
    _, predicted = torch.max(y_pred, 1)
    return (predicted == y_true).float().mean().item()

# Early stopping parameters
patience = 5  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
early_stop_counter = 0
early_stopping = False

# Train the model with early stopping
num_epochs = 50
for epoch in range(num_epochs):
    if early_stopping:
        print("Early stopping triggered. Stopping training.")
        break

    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    # Training loop with progress bar
    progress_bar = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for i, (images, labels) in enumerate(progress_bar):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update running loss and accuracy
        running_loss += loss.item()
        correct_predictions += (outputs.argmax(1) == labels).sum().item()
        total_predictions += labels.size(0)

        # Update progress bar description
        progress_bar.set_postfix({
            'loss': running_loss / (i + 1),
            'accuracy': correct_predictions / total_predictions
        })
        # Validation step
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            correct_val_predictions += (outputs.argmax(1) == labels).sum().item()
            total_val_predictions += labels.size(0)

    # Calculate validation metrics
    val_loss /= len(val_loader)
    val_accuracy = correct_val_predictions / total_val_predictions

    # Print epoch summary
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {correct_predictions / total_predictions:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0  # Reset counter if validation loss improves
        torch.save(model.state_dict(), 'best_pest_detection_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        print(f"Early stopping counter: {early_stop_counter}/{patience}")
        if early_stop_counter >= patience:
            print("No improvement for several epochs. Triggering early stopping.")
            early_stopping = True

C:\Users\sophi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sophi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/50: 100%|█████████████████████████████████████████| 194/194 [42:36<00:00, 13.18s/it, loss=1.02, accuracy=0.677]


Epoch 1/50
Train Loss: 1.0205, Train Accuracy: 0.6770, Val Loss: 0.8569, Val Accuracy: 0.7203


Epoch 2/50: 100%|█████████████████████████████████████████| 194/194 [55:01<00:00, 17.02s/it, loss=0.496, accuracy=0.84]


Epoch 2/50
Train Loss: 0.4960, Train Accuracy: 0.8402, Val Loss: 1.0496, Val Accuracy: 0.7087
Early stopping counter: 1/5


Epoch 3/50: 100%|████████████████████████████████████████| 194/194 [29:05<00:00,  9.00s/it, loss=0.287, accuracy=0.907]


...

Epoch 10/50
Train Loss: 0.0970, Train Accuracy: 0.9691, Val Loss: 0.3421, Val Accuracy: 0.9134
Early stopping counter: 3/5


Epoch 11/50: 100%|████████████████████████████████████| 194/194 [1:25:38<00:00, 26.49s/it, loss=0.0903, accuracy=0.972]


Epoch 11/50
Train Loss: 0.0903, Train Accuracy: 0.9716, Val Loss: 0.3910, Val Accuracy: 0.8882
Early stopping counter: 4/5


Epoch 12/50: 100%|████████████████████████████████████████| 194/194 [37:35<00:00, 11.63s/it, loss=0.098, accuracy=0.97]


Epoch 12/50
Train Loss: 0.0980, Train Accuracy: 0.9703, Val Loss: 0.4177, Val Accuracy: 0.9031
Early stopping counter: 5/5
No improvement for several epochs. Triggering early stopping.
Early stopping triggered. Stopping training.


In [15]:
# Save the trained model
torch.save(model.state_dict(), 'pest_detection_model.pth')

NameError: name 'model' is not defined

In [23]:
# Function to evaluate the model on a given dataset
def evaluate_model(loader, model, criterion, device):
    model.eval()
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
             # Calculate loss
            total_loss += loss.item()

            # Calculate accuracy
            correct_predictions += (outputs.argmax(1) == labels).sum().item()
            total_predictions += labels.size(0)

    avg_loss = total_loss / len(loader)
    accuracy = correct_predictions / total_predictions

    return avg_loss, accuracy

# Evaluate on training data
train_loss, train_accuracy = evaluate_model(train_loader, model, criterion, device)
print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

Train Loss: 0.0833, Train Accuracy: 0.9743


In [24]:
# Evaluate on validation data
val_loss, val_accuracy = evaluate_model(val_loader, model, criterion, device)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Validation Loss: 0.0824, Validation Accuracy: 0.9767


In [14]:
!pip install scikit-learn


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.1 MB 2.2 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.1 MB 2.4 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.1 MB 2.4 MB/s eta 0:00:04
   ---------- ----------------------------- 2.9/11.1 MB 2.6 MB/s eta 0:00:04
   ------------- -------------------------- 3.7/11.1 MB 2.9 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.1 MB 3.1 MB/s eta 0:00:03
   ------------------- -------------------- 5.5/11.1 MB 3.2 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.1 MB 3.3 MB/s eta 0:00:02
   ------------------------- ------

In [25]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Assuming you have a dataset object like `train_dataset`
dataset_size = len(train_dataset)
indices = list(range(dataset_size))

# Split the indices into training and test sets (e.g., 80% train, 20% test)
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Create subsets for train and test data
train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(train_dataset, test_indices)# Create DataLoader for train and test subsets
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

In [16]:
# Evaluate on test data
test_loss, test_accuracy = evaluate_model(test_loader, model, criterion, device)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.0876, Test Accuracy: 0.9766


In [37]:
import requests
from PIL import Image
from io import BytesIO
import torch
from torchvision import transforms

# Function to download and preprocess the image
def preprocess_image_from_url(url, transform):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Define your image transformation pipeline (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Example URL of the image
image_url = "https://th.bing.com/th/id/OIP.DHzBfQKXXFZj8o70V0EjSgHaF9?rs=1&pid=ImgDetMain"
image_tensor = preprocess_image_from_url(image_url, transform)


In [38]:
# Assuming your model is already defined and loaded
model.eval()  # Set the model to evaluation mode

# Move image tensor to the appropriate device
image_tensor = image_tensor.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(image_tensor)

# Process outputs to get the predicted class
_, predicted_class = torch.max(outputs, 1)
print(f"Predicted class: {predicted_class.item()}")


Predicted class: 10
